<a href="https://colab.research.google.com/github/gabrielcampos21/ETL-com-Python/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [77]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [78]:
import pandas as pd

# lista de usuários que terão mensagens personalizadas criadas
df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5]


In [79]:
import requests, json

users = []

# função para obter dados de determinado usuario
def get_user_data(user_id):
  response = requests.get(f'{sdw2023_api_url}/users/{user_id}')
  return response.json() if response.status_code == 200 else None

# populando lista de usuarios com seus dados
for user_id in user_ids:
  data = get_user_data(user_id)
  if data != None:
    users.append(data)

# função para salvar a lista de usuarios em um arquivo csv
def salvar_dados(lista_de_usuarios, nome_arquivo = "users_data.csv"):
  # salvando em um arquivo csv (users_data.csv)
  users_df = pd.DataFrame(lista_de_usuarios)
  users_df.to_csv(nome_arquivo, encoding="utf-8", index=False)
  print(f"Dados de {len(lista_de_usuarios)} usuários foram salvos em {nome_arquivo}")

salvar_dados(users)

Dados de 2 usuários foram salvos em users_data.csv


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [80]:
# criar o csv com as frases, caso não exista

import os

arquivo_frases = "frases.csv"

if not os.path.exists(arquivo_frases):
  # frases geradas pelo ChatGPT
  frases = [
    "Olá {user}, investir é uma maneira inteligente de construir seu futuro financeiro.",
    "{user}, com investimentos adequados, você pode alcançar suas metas financeiras mais rapidamente.",
    "Diversificar seus investimentos, {user}, é chave para reduzir riscos e aumentar oportunidades.",
    "Seu patrimônio crescerá ao longo do tempo, {user}, quando você escolhe investir de forma estratégica.",
    "Planejar o futuro é essencial, {user}, e investir faz parte desse plano para garantir estabilidade.",
    "Aproveite os juros compostos, {user}, ao investir - isso significa ver seu dinheiro crescer ainda mais.",
    "Para investir de maneira eficaz, é importante conhecer seu perfil de investidor, {user}.",
    "Como {user} interessado em investir, você encontrará opções alinhadas às suas necessidades no mercado.",
    "Fique atualizado sobre a economia, {user}, para tomar decisões informadas sobre seus investimentos.",
    "Começar a investir mais cedo, {user}, dá a você a vantagem do tempo e do potencial de crescimento.",
    "Lembre-se, {user}, de revisar seus investimentos periodicamente para garantir que continuem atendendo seus objetivos.",
    "Olá {user}, imagine alcançar seus sonhos financeiros por meio de investimentos inteligentes.",
    "{user}, investir é como plantar sementes financeiras que crescerão com o tempo.",
    "Investir bem é como construir bases sólidas para a realização dos seus objetivos, {user}.",
    "Com o planejamento certo, {user}, você pode garantir estabilidade financeira por meio de investimentos.",
    "A estratégia de investimento certa pode transformar seus planos em realidade, {user}.",
    "Lembre-se, {user}, investir não é apenas para os ricos - é uma jornada para todos que buscam crescimento financeiro.",
    "Ao investir, {user}, você está tomando medidas proativas para garantir um futuro financeiro seguro.",
    "Investir sabiamente, {user}, é uma expressão tangível de cuidado com o seu próprio futuro.",
    "Seja o protagonista da sua jornada financeira, {user}, investindo para conquistar independência e tranquilidade.",
    "{user}, suas escolhas de investimento hoje moldarão a qualidade de vida que você terá amanhã.",
    "Assim como cada gota faz um oceano, cada investimento o aproxima de seus objetivos, {user}.",
  ]

  # salvando em formato csv
  frases_df = pd.DataFrame(frases)
  frases_df.to_csv("frases.csv", encoding="utf-8", index=False)
  print("Arquivo de frases criado!")

else:
  # obter lista de frases do arquivo frases.csv, geradas pelo ChatGPT
  # no caso do arquivo csv já existir
  print("Arquivo de frases encontrado!")
  frases = pd.read_csv("frases.csv")
  frases = frases["0"].to_list()



Arquivo de frases criado!


In [81]:
import random

# função para gerar frase com o nome do usuário
def gerar_frase(nome_do_usuario):
  return random.choice(frases).replace("{user}", nome_do_usuario)

In [82]:
# gerar frases personalizadas para cada usuário
for user in users:
  news = gerar_frase(user['name'])
  print(f"frase personalizada criada: {news}")
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

frase personalizada criada: Seu patrimônio crescerá ao longo do tempo, Devweekerson, quando você escolhe investir de forma estratégica.
frase personalizada criada: Pip, com investimentos adequados, você pode alcançar suas metas financeiras mais rapidamente.


## **L**oad

Atualize a lista de "news" de cada usuário com a nova mensagem gerada.

In [83]:
# salvar em arquivo csv os dados atualizados
salvar_dados(users)

Dados de 2 usuários foram salvos em users_data.csv
